In [1]:
import os
from pathlib import Path
import numpy as np 
import pandas as pd 
pd.set_option("display.float_format", "{:.4f}".format)
base_path = Path.cwd().parent
os.chdir(base_path) 

Objective

1.  Calculate the portfolio’s monthly and yearly returns as a percentage based on dollar Profit and Loss (PnL) while accounting for:
    - New allocations (subscriptions)
    - redemptions (withdrawals)

2. Attribute returns across five asset classes: Equity, Rates, Commodity, FX, and Credit 

In [2]:
bom_aum = pd.read_excel(r"data/Week1 Input Data 2024.xlsx", sheet_name="BOM AUM")
allocation = pd.read_excel(r"data/Week1 Input Data 2024.xlsx", sheet_name="Allocation")
asset_pnl = pd.read_excel(r"data/Week1 Input Data 2024.xlsx", sheet_name="PnL by Asset Class")
asset_pnl["Date"] = pd.to_datetime(asset_pnl["Date"], format = "%m/%d/%Y")

Step 1 BOD AUM (Beginning of Day AUM): 

1. You need to determine the Beginning of Day AUM for each day, considering the AUM from the previous day, any capital allocation activities (subscriptions or redemptions), and the daily PnL


In [3]:
def get_bod_aum(asset_pnl_df: pd.DataFrame, allocation_df: pd.DataFrame, bom_aum_df: pd.DataFrame):

    # Aggregate daily change for all asset class
    daily_change = asset_pnl_df.groupby("Date").agg( Daily_PnL_Sum = ("Daily PnL" , "sum")).reset_index()
    
    # Attach allocation and bom_aum to the df
    update_allocation = pd.merge(daily_change, allocation_df, on = "Date", how = "left").fillna(0)
    update_bom_aum = pd.merge(update_allocation, bom_aum_df, on = "Date", how = "left").fillna(0)

    # Identify YM
    update_bom_aum["YM"] = update_bom_aum["Date"].apply(lambda x : f"{x.year}{x.month}")

    # Get EOD AUM
    update_bom_aum["tmp"] = update_bom_aum['Daily_PnL_Sum'] + update_bom_aum['Allocation'] + update_bom_aum['BOM AUM']
    update_bom_aum["EOD AUM"] = update_bom_aum.groupby("YM")["tmp"].cumsum()

    # Get BOD AUM
    update_bom_aum["BOD AUM"] = update_bom_aum["EOD AUM"].shift(1).fillna(0)

    # Change BOD AUM for first day of month to the reconcile number
    update_bom_aum["BOD AUM"] = np.where(update_bom_aum["BOM AUM"]!= 0,update_bom_aum["BOM AUM"],update_bom_aum["BOD AUM"])


    return update_bom_aum [["YM", "Date", "Daily_PnL_Sum", "Allocation", "BOD AUM"]]

In [4]:
bod_aum_portfolio = get_bod_aum(asset_pnl,allocation,bom_aum)
bod_aum_portfolio.head(5)

,YM,Date,Daily_PnL_Sum,Allocation,BOD AUM
0,20241,2024-01-02,235000,0.0000,200000000.0000
1,20241,2024-01-03,-1065000,0.0000,200235000.0000
2,20241,2024-01-04,-592000,0.0000,199170000.0000
3,20241,2024-01-05,1323000,0.0000,198578000.0000
4,20241,2024-01-08,1470000,0.0000,199901000.0000


In [5]:
bod_aum_portfolio.groupby("YM").agg({"Date":"min",
                           "BOD AUM": "first"})

,Date,BOD AUM
YM,,
20241,2024-01-02,200000000.0000
20242,2024-02-01,300000000.0000
20243,2024-03-01,250000000.0000
20244,2024-04-01,200000000.0000
20245,2024-05-01,300000000.0000
20246,2024-06-03,300000000.0000
20247,2024-07-01,200000000.0000
20248,2024-08-01,200000000.0000


Step 2 Daily Return Calculation: 

2. Use the daily PnL by asset class and the Beginning of Day AUM to calculate both the daily return for each asset class and the daily return for the portfolio.

In [6]:
# Daily return for the portfolio
bod_aum_portfolio["Daily_Return_Tot"] = bod_aum_portfolio["Daily_PnL_Sum"] / bod_aum_portfolio["BOD AUM"]
bod_aum_portfolio

,YM,Date,Daily_PnL_Sum,Allocation,BOD AUM,Daily_Return_Tot
0,20241,2024-01-02,235000,0.0000,200000000.0000,0.0012
1,20241,2024-01-03,-1065000,0.0000,200235000.0000,-0.0053
2,20241,2024-01-04,-592000,0.0000,199170000.0000,-0.0030
3,20241,2024-01-05,1323000,0.0000,198578000.0000,0.0067
4,20241,2024-01-08,1470000,0.0000,199901000.0000,0.0074
...,...,...,...,...,...,...
162,20248,2024-08-26,467000,0.0000,259978000.0000,0.0018
163,20248,2024-08-27,157000,0.0000,260445000.0000,0.0006
164,20248,2024-08-28,-801000,0.0000,260602000.0000,-0.0031
165,20248,2024-08-29,-188000,0.0000,259801000.0000,-0.0007


In [7]:
# Daily return for each asset class
bod_aum_asset = pd.merge(asset_pnl, bod_aum_portfolio[["YM", "Date", "BOD AUM", "Daily_PnL_Sum", "Daily_Return_Tot"]], how = "left", on = "Date")
bod_aum_asset["Daily_Return_Asset"] =  bod_aum_asset["Daily_Return_Tot"] * bod_aum_asset["Daily PnL"] / bod_aum_asset["Daily_PnL_Sum"]
bod_aum_asset

,Date,Asset Class,Daily PnL,YM,BOD AUM,Daily_PnL_Sum,Daily_Return_Tot,Daily_Return_Asset
0,2024-01-02,Commodity,68000,20241,200000000.0000,235000,0.0012,0.0003
1,2024-01-03,Commodity,429000,20241,200235000.0000,-1065000,-0.0053,0.0021
2,2024-01-04,Commodity,-56000,20241,199170000.0000,-592000,-0.0030,-0.0003
3,2024-01-05,Commodity,278000,20241,198578000.0000,1323000,0.0067,0.0014
4,2024-01-08,Commodity,-693000,20241,199901000.0000,1470000,0.0074,-0.0035
...,...,...,...,...,...,...,...,...
830,2024-08-26,Rates,328000,20248,259978000.0000,467000,0.0018,0.0013
831,2024-08-27,Rates,-460000,20248,260445000.0000,157000,0.0006,-0.0018
832,2024-08-28,Rates,-480000,20248,260602000.0000,-801000,-0.0031,-0.0018
833,2024-08-29,Rates,221000,20248,259801000.0000,-188000,-0.0007,0.0009


Step 3 Monthly Return Calculation: 

3. Calculate the monthly returns independently for each month. This allows you to assess each month’s performance in isolation. The monthly return of the portfolio should be calculated by compounding the daily returns of the portfolio for that month

In [8]:
monthly_return_portfolio = bod_aum_portfolio.groupby("YM").agg(
    BOM_AUM = ("BOD AUM", "first"),
    Monthly_PnL_Sum = ("Daily_PnL_Sum","sum"),
    monthly_return = ("Daily_Return_Tot", lambda x: ((1+x).cumprod().iloc[-1]) - 1)
    ).reset_index()
monthly_return_portfolio

,YM,BOM_AUM,Monthly_PnL_Sum,monthly_return
0,20241,200000000.0000,9997000,0.0451
1,20242,300000000.0000,8007000,0.0267
2,20243,250000000.0000,14951000,0.0553
3,20244,200000000.0000,5810000,0.0262
4,20245,300000000.0000,6255000,0.0209
5,20246,300000000.0000,4398000,0.0149
6,20247,200000000.0000,2982000,0.0149
7,20248,200000000.0000,9724000,0.0504


Step 4 Monthly Return Attribution: 

4. To attribute the monthly return of the portfolio to each asset class, 
    - Assume an initial hypothetical AUM of 100 at the beginning of the month for the entire portfolio. 
    - Then, calculate the hypothetical daily PnL for each asset class throughout the month (using the daily return data for each asset class). 
    - The sum of the cumulative hypothetical daily PnL for each asset class should match the total portfolio monthly return to ensure consistency and traceability

In [9]:
# Transposing "Asset Class" into columns
bod_aum_asset_pivot = bod_aum_asset.groupby(["YM", "Date", "Asset Class"]).agg({"Daily_Return_Asset": "first"}).reset_index()
bod_aum_asset_pivot = bod_aum_asset_pivot.pivot(index=["YM", "Date"], columns="Asset Class", values="Daily_Return_Asset").reset_index()
bod_aum_asset_pivot.columns.name = None 

# Sum up the returns from each asset class
bod_aum_asset_pivot["Return_Tot"] = bod_aum_asset_pivot.apply(lambda row : sum(row[col] for col in ['Commodity', 'Credit', 'Equity', 'FX', 'Rates']), axis = 1)

# Calculate the cumulative return for each month 
bod_aum_asset_pivot["Return_Tot_Cum"] = bod_aum_asset_pivot.groupby("YM")["Return_Tot"].transform(lambda x : (1+x).cumprod())

# Calculate the EOD AUM
bod_aum_asset_pivot["EOD AUM"] = 100*bod_aum_asset_pivot["Return_Tot_Cum"] 

# Use shift to get the BOD AUM
bod_aum_asset_pivot["BOD AUM"] = bod_aum_asset_pivot.groupby("YM")["EOD AUM"].transform(lambda x : x.shift(1)).fillna(100)

# Calculate the hypothetical daily PnL for each asset class
for col in ['Commodity', 'Credit', 'Equity', 'FX', 'Rates']:
    bod_aum_asset_pivot[f"{col} Hypo PnL"] = bod_aum_asset_pivot.apply(lambda row:row[col]*row["BOD AUM"], axis = 1)

# Calculate the Hypothetical PnL for the portfolio 
# bod_aum_asset_pivot["Portfolio Hypo PnL"] = bod_aum_asset_pivot["EOD AUM"] - bod_aum_asset_pivot["BOD AUM"]
bod_aum_asset_pivot["Portfolio Hypo PnL"] = bod_aum_asset_pivot.apply(lambda row : sum(row[col] for col in [i + ' Hypo PnL' for i in ['Commodity', 'Credit', 'Equity', 'FX', 'Rates']]), axis = 1)

In [10]:
bod_aum_asset_pivot.head(5)

,YM,Date,Commodity,Credit,Equity,FX,Rates,Return_Tot,Return_Tot_Cum,EOD AUM,BOD AUM,Commodity Hypo PnL,Credit Hypo PnL,Equity Hypo PnL,FX Hypo PnL,Rates Hypo PnL,Portfolio Hypo PnL
0,20241,2024-01-02,0.0003,0.0002,-0.0010,0.0006,0.0010,0.0012,1.0012,100.1175,100.0000,0.0340,0.0195,-0.1010,0.0600,0.1050,0.1175
1,20241,2024-01-03,0.0021,-0.0005,-0.0074,0.0004,0.0001,-0.0053,0.9958,99.5850,100.1175,0.2145,-0.0545,-0.7405,0.0380,0.0100,-0.5325
2,20241,2024-01-04,-0.0003,-0.0008,-0.0010,-0.0001,-0.0008,-0.0030,0.9929,99.2890,99.5850,-0.0280,-0.0780,-0.1030,-0.0060,-0.0810,-0.2960
3,20241,2024-01-05,0.0014,0.0002,0.0056,-0.0000,-0.0005,0.0067,0.9995,99.9505,99.2890,0.1390,0.0155,0.5550,-0.0015,-0.0465,0.6615
4,20241,2024-01-08,-0.0035,0.0011,0.0092,-0.0003,0.0008,0.0074,1.0069,100.6855,99.9505,-0.3465,0.1120,0.9235,-0.0295,0.0755,0.7350


In [11]:
# Reconciliation
bod_aum_asset_pivot.groupby("YM")["Portfolio Hypo PnL"].sum()

YM
20241   4.5098
20242   2.6690
20243   5.5342
20244   2.6171
20245   2.0850
20246   1.4936
20247   1.4910
20248   5.0442
Name: Portfolio Hypo PnL, dtype: float64

In [12]:
monthly_return_portfolio["monthly_return"]*100

0   4.5098
1   2.6690
2   5.5342
3   2.6171
4   2.0850
5   1.4936
6   1.4910
7   5.0442
Name: monthly_return, dtype: float64

Step 5 Yearly Return Calculation:

5. Yearly Return should be calculated by compounding monthly return. 
- Use the same hypothetical PnL approach for calculating the yearly return attributions. 
- Leverage the monthly returns by asset class and the monthly portfolio returns to calculate the overall yearly performance.  

In [13]:
yearly_return_portfolio = monthly_return_portfolio["monthly_return"].apply(lambda x : 1+x).cumprod().iloc[-1]
yearly_return_portfolio

1.2835403713555786

In [14]:
bod_aum_asset["Year"] = bod_aum_asset["Date"].dt.year

bom_aum_asset = bod_aum_asset.groupby(["Year","YM","Asset Class"]).agg(
    BOM_AUM = ("BOD AUM", "first"),
    Monthly_PNL = ("Daily PnL","sum")
    ).reset_index()

bom_aum_asset = pd.merge(bom_aum_asset, monthly_return_portfolio[["YM", "Monthly_PnL_Sum","monthly_return"]], how = "left", on = "YM")

bom_aum_asset["Monthly_Return_Asset"] = bom_aum_asset["monthly_return"]  * bom_aum_asset["Monthly_PNL"]  / bom_aum_asset["Monthly_PnL_Sum"] 

In [17]:
bom_aum_asset.head(2)

,Year,YM,Asset Class,BOM_AUM,Monthly_PNL,Monthly_PnL_Sum,monthly_return,Monthly_Return_Asset
0,2024,20241,Commodity,200000000.0000,2484000,9997000,0.0451,0.0112
1,2024,20241,Credit,200000000.0000,223000,9997000,0.0451,0.0010


In [18]:
bom_aum_asset_pivot = bom_aum_asset.groupby(["Year","YM", "Asset Class"]).agg({"Monthly_Return_Asset": "first"}).reset_index()
bom_aum_asset_pivot = bom_aum_asset_pivot.pivot(index=["Year","YM"], columns="Asset Class", values="Monthly_Return_Asset").reset_index()
bom_aum_asset_pivot.columns.name = None 

# Sum up the returns from each asset class
bom_aum_asset_pivot["Return_Tot"] = bom_aum_asset_pivot.apply(lambda row : sum(row[col] for col in ['Commodity', 'Credit', 'Equity', 'FX', 'Rates']), axis = 1)

# Calculate the cumulative return for each month 
bom_aum_asset_pivot["Return_Tot_Cum"] = bom_aum_asset_pivot.groupby("Year")["Return_Tot"].transform(lambda x : (1+x).cumprod())

# Calculate the EOD AUM
bom_aum_asset_pivot["EOD AUM"] = 100*bom_aum_asset_pivot["Return_Tot_Cum"] 

# Use shift to get the BOD AUM
bom_aum_asset_pivot["BOD AUM"] = bom_aum_asset_pivot.groupby("Year")["EOD AUM"].transform(lambda x : x.shift(1)).fillna(100)


# Calculate the hypothetical daily PnL for each asset class
for col in ['Commodity', 'Credit', 'Equity', 'FX', 'Rates']:
    bom_aum_asset_pivot[f"{col} Hypo PnL"] = bom_aum_asset_pivot.apply(lambda row:row[col]*row["BOD AUM"], axis = 1)

# Calculate the Hypothetical PnL for the portfolio 
# bod_aum_asset_pivot["Portfolio Hypo PnL"] = bod_aum_asset_pivot["EOD AUM"] - bod_aum_asset_pivot["BOD AUM"]
bom_aum_asset_pivot["Portfolio Hypo PnL"] = bom_aum_asset_pivot.apply(lambda row : sum(row[col] for col in [i + ' Hypo PnL' for i in ['Commodity', 'Credit', 'Equity', 'FX', 'Rates']]), axis = 1)

bom_aum_asset_pivot

,Year,YM,Commodity,Credit,Equity,FX,Rates,Return_Tot,Return_Tot_Cum,EOD AUM,BOD AUM,Commodity Hypo PnL,Credit Hypo PnL,Equity Hypo PnL,FX Hypo PnL,Rates Hypo PnL,Portfolio Hypo PnL
0,2024,20241,0.0112,0.0010,0.0271,0.0033,0.0024,0.0451,1.0451,104.5098,100.0000,1.1206,0.1006,2.7130,0.3311,0.2445,4.5098
1,2024,20242,0.0033,0.0004,0.0239,0.0008,-0.0016,0.0267,1.0730,107.2992,104.5098,0.3400,0.0435,2.4960,0.0791,-0.1693,2.7894
2,2024,20243,0.0170,0.0073,0.0250,0.0039,0.0022,0.0553,1.1324,113.2373,107.2992,1.8230,0.7860,2.6841,0.4143,0.2308,5.9381
3,2024,20244,0.0056,0.0021,0.0212,0.0022,-0.0049,0.0262,1.1620,116.2009,113.2373,0.6340,0.2402,2.3969,0.2520,-0.5596,2.9636
4,2024,20245,0.0016,0.0024,0.0158,-0.0008,0.0019,0.0209,1.1862,118.6237,116.2009,0.1817,0.2766,1.8367,-0.0918,0.2196,2.4228
5,2024,20246,0.0018,0.0008,0.0080,0.0019,0.0025,0.0149,1.2040,120.3954,118.6237,0.2119,0.0983,0.9483,0.2200,0.2933,1.7717
6,2024,20247,-0.0097,0.0034,0.0164,0.0006,0.0042,0.0149,1.2219,122.1905,120.3954,-1.1708,0.4136,1.9745,0.0686,0.5093,1.7951
7,2024,20248,0.0025,0.0014,0.0522,-0.0072,0.0015,0.0504,1.2835,128.3540,122.1905,0.3042,0.1756,6.3784,-0.8804,0.1857,6.1635


In [19]:
# Reconciliation
bom_aum_asset_pivot.groupby("Year")["Portfolio Hypo PnL"].sum()

Year
2024   28.3540
Name: Portfolio Hypo PnL, dtype: float64

In [20]:
yearly_return_portfolio

1.2835403713555786

Step 6 Report:

6. Report results
- Portfolio's monthly return and yearly return in percentage
- Attribute the monthly return to each asset class. 

In [29]:
monthly_return_portfolio["monthly_return_percent"] = monthly_return_portfolio["monthly_return"].apply(lambda x: f"{round(100*x,2)}%")
print(f"Monthly Return:\n{monthly_return_portfolio[['YM', 'monthly_return_percent']]}")
print(f"Yearly Return: {round((yearly_return_portfolio-1)*100,2)}%")


Monthly Return:
      YM monthly_return_percent
0  20241                  4.51%
1  20242                  2.67%
2  20243                  5.53%
3  20244                  2.62%
4  20245                  2.09%
5  20246                  1.49%
6  20247                  1.49%
7  20248                  5.04%
Yearly Return: 28.35%


In [37]:
monthly_return_allocation = bom_aum_asset_pivot.copy()
for col in ['Commodity', 'Credit', 'Equity', 'FX', 'Rates', 'Return_Tot']:
    monthly_return_allocation[f'{col}%'] = monthly_return_allocation[col].apply(lambda x : f'{round(x*100,2)}%')
monthly_return_allocation[["Year", "YM"] + [i+"%" for i in ['Commodity', 'Credit', 'Equity', 'FX', 'Rates', 'Return_Tot']]]

,Year,YM,Commodity%,Credit%,Equity%,FX%,Rates%,Return_Tot%
0,2024,20241,1.12%,0.1%,2.71%,0.33%,0.24%,4.51%
1,2024,20242,0.33%,0.04%,2.39%,0.08%,-0.16%,2.67%
2,2024,20243,1.7%,0.73%,2.5%,0.39%,0.22%,5.53%
3,2024,20244,0.56%,0.21%,2.12%,0.22%,-0.49%,2.62%
4,2024,20245,0.16%,0.24%,1.58%,-0.08%,0.19%,2.09%
5,2024,20246,0.18%,0.08%,0.8%,0.19%,0.25%,1.49%
6,2024,20247,-0.97%,0.34%,1.64%,0.06%,0.42%,1.49%
7,2024,20248,0.25%,0.14%,5.22%,-0.72%,0.15%,5.04%
